<a href="https://colab.research.google.com/github/meligurevich/IAPH/blob/main/Desafio_3GUREVICH.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MELISA SABRINA GUREVICH

DNI: 35.365.884

### Vectorización de texto y modelo de clasificación Naïve Bayes con el dataset 20 newsgroups

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score

# 20newsgroups por ser un dataset clásico de NLP ya viene incluido y formateado
# en sklearn
from sklearn.datasets import fetch_20newsgroups
import numpy as np

## Carga de datos

In [ ]:
# cargamos los datos (ya separados de forma predeterminada en train y test)
newsgroups_train = fetch_20newsgroups(subset='train', remove=('headers', 'footers', 'quotes'))
newsgroups_test = fetch_20newsgroups(subset='test', remove=('headers', 'footers', 'quotes'))

## Vectorización

In [ ]:
# instanciamos un vectorizador
# ver diferentes parámetros de instanciación en la documentación de sklearn
tfidfvect = TfidfVectorizer()

In [ ]:
# en el atributo `data` accedemos al texto
newsgroups_train.data[0]

'I was wondering if anyone out there could enlighten me on this car I saw\nthe other day. It was a 2-door sports car, looked to be from the late 60s/\nearly 70s. It was called a Bricklin. The doors were really small. In addition,\nthe front bumper was separate from the rest of the body. This is \nall I know. If anyone can tellme a model name, engine specs, years\nof production, where this car is made, history, or whatever info you\nhave on this funky looking car, please e-mail.'

In [ ]:
# con la interfaz habitual de sklearn podemos fitear el vectorizador
# (obtener el vocabulario y calcular el vector IDF)
# y transformar directamente los datos
X_train = tfidfvect.fit_transform(newsgroups_train.data)
# `X_train` la podemos denominar como la matriz documento-término

In [ ]:
# recordar que las vectorizaciones por conteos son esparsas
# por ello sklearn convenientemente devuelve los vectores de documentos
# como matrices esparsas
print(type(X_train))
print(f'shape: {X_train.shape}')
print(f'cantidad de documentos: {X_train.shape[0]}')
print(f'tamaño del vocabulario (dimensionalidad de los vectores): {X_train.shape[1]}')

<class 'scipy.sparse._csr.csr_matrix'>
shape: (11314, 101631)
cantidad de documentos: 11314
tamaño del vocabulario (dimensionalidad de los vectores): 101631


In [ ]:
# una vez ajustado el vectorizador, podemos acceder a atributos como el vocabulario
# aprendido. Es un diccionario que va de términos a índices.
# El índice es la posición en el vector de documento.
tfidfvect.vocabulary_['car']

25775

In [ ]:
# es muy útil tener el diccionario opuesto que va de índices a términos
idx2word = {v: k for k,v in tfidfvect.vocabulary_.items()}

In [ ]:
# en `y_train` guardamos los targets que son enteros
y_train = newsgroups_train.target
y_train[:10]

array([ 7,  4,  4,  1, 14, 16, 13,  3,  2,  4])

In [ ]:
# hay 20 clases correspondientes a los 20 grupos de noticias
print(f'clases {np.unique(newsgroups_test.target)}')
newsgroups_test.target_names

clases [ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19]


['alt.atheism',
 'comp.graphics',
 'comp.os.ms-windows.misc',
 'comp.sys.ibm.pc.hardware',
 'comp.sys.mac.hardware',
 'comp.windows.x',
 'misc.forsale',
 'rec.autos',
 'rec.motorcycles',
 'rec.sport.baseball',
 'rec.sport.hockey',
 'sci.crypt',
 'sci.electronics',
 'sci.med',
 'sci.space',
 'soc.religion.christian',
 'talk.politics.guns',
 'talk.politics.mideast',
 'talk.politics.misc',
 'talk.religion.misc']

## Similaridad de documentos

In [ ]:
# Veamos similaridad de documentos. Tomemos algún documento
idx = 4811
print(newsgroups_train.data[idx])

THE WHITE HOUSE

                  Office of the Press Secretary
                   (Pittsburgh, Pennslyvania)
______________________________________________________________
For Immediate Release                         April 17, 1993     

             
                  RADIO ADDRESS TO THE NATION 
                        BY THE PRESIDENT
             
                Pittsburgh International Airport
                    Pittsburgh, Pennsylvania
             
             
10:06 A.M. EDT
             
             
             THE PRESIDENT:  Good morning.  My voice is coming to
you this morning through the facilities of the oldest radio
station in America, KDKA in Pittsburgh.  I'm visiting the city to
meet personally with citizens here to discuss my plans for jobs,
health care and the economy.  But I wanted first to do my weekly
broadcast with the American people. 
             
             I'm told this station first broadcast in 1920 when
it reported that year's presidential elec

In [ ]:
# midamos la similaridad coseno con todos los documentos de train
cossim = cosine_similarity(X_train[idx], X_train)[0]

In [ ]:
cossim

array([0.1382319 , 0.1067036 , 0.23029327, ..., 0.12320753, 0.08765353,
       0.04415046])

In [ ]:
# podemos ver los valores de similaridad ordenados de mayor a menos
np.sort(cossim)[::-1]

array([1.        , 0.70930477, 0.67474953, ..., 0.        , 0.        ,
       0.        ])

In [ ]:
# y a qué documentos corresponden
np.argsort(cossim)[::-1]

array([4811, 6635, 4253, ..., 9019, 9016, 8748])

In [ ]:
# los 5 documentos más similares:
mostsim = np.argsort(cossim)[::-1][1:6]

In [ ]:
# el documento original pertenece a la clase:
newsgroups_train.target_names[y_train[idx]]

'talk.politics.misc'

In [ ]:
# y los 5 más similares son de las clases:
for i in mostsim:
  print(newsgroups_train.target_names[y_train[i]])

talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc
talk.politics.misc


### Modelo de clasificación Naïve Bayes

In [ ]:
# es muy fácil instanciar un modelo de clasificación Naïve Bayes y entrenarlo con sklearn
clf = MultinomialNB()
clf.fit(X_train, y_train)

MultinomialNB()

In [ ]:
# con nuestro vectorizador ya fiteado en train, vectorizamos los textos
# del conjunto de test
X_test = tfidfvect.transform(newsgroups_test.data)
y_test = newsgroups_test.target
y_pred =  clf.predict(X_test)

In [ ]:
# el F1-score es una metrica adecuada para reportar desempeño de modelos de claificación
# es robusta al desbalance de clases. El promediado 'macro' es el promedio de los
# F1-score de cada clase. El promedio 'micro' es equivalente a la accuracy que no
# es una buena métrica cuando los datasets son desbalanceados
f1_score(y_test, y_pred, average='macro')

0.5854345727938506

### Consigna del desafío 2

Alumna: Melisa Sabrina Gurevich

**1**. Vectorizar documentos. Tomar 5 documentos al azar y medir similaridad con el resto de los documentos.
Estudiar los 5 documentos más similares de cada uno analizar si tiene sentido
la similaridad según el contenido del texto y la etiqueta de clasificación.



In [ ]:
# Impprtamos librerías necesarias
import pandas as pd
import numpy as np
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Cargamos dataset
categories = ['rec.sport.hockey', 'sci.space', 'talk.politics.misc', 'comp.graphics']
newsgroups = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))

df = pd.DataFrame({
    'text': newsgroups.data,
    'label': [newsgroups.target_names[i] for i in newsgroups.target]
})

# Vectorizamos textos con CountVectorizer
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(df['text'])

# Seleccionamos 5 documentos al azar
np.random.seed(42)
selected_indices = np.random.choice(len(df), size=5, replace=False)

# Calculamos similitud coseno entre documentos
similarity_matrix = cosine_similarity(X)

# Para cada documento, encontrar los 5 más similares
for idx in selected_indices:
    print(f"\nDocumento original (índice {idx}) - Categoría: {df.iloc[idx]['label']}")
    print(df.iloc[idx]['text'][:300], "...")  # Mostrar primeros 300 caracteres

    # Calculamos similitud con el resto
    sim_scores = similarity_matrix[idx]
    sim_scores[idx] = -1  # evitar seleccionar el mismo

    # Obtenemos índices de los 5 más similares
    top5_idx = np.argsort(sim_scores)[-5:][::-1]

    print("\nDocumentos más similares:")
    for i, sim_idx in enumerate(top5_idx):
        print(f"\n{i+1}. Similitud: {sim_scores[sim_idx]:.3f} - Categoría: {df.iloc[sim_idx]['label']}")
        print(df.iloc[sim_idx]['text'][:200], "...")



Documento original (índice 727) - Categoría: comp.graphics
Does anyone know the phone number to a place where i can get
a VGA passthrough?

	I want to hook up my VGA card to my XGA card (whcih you can can).
All I need is the cable that connects them.  It is the same type of
cable that you would connect from your VGA card to say a Video Blaster
or something. ...

Documentos más similares:

1. Similitud: 0.574 - Categoría: comp.graphics
Does anyone know of a VL-Bus video card based on the ET4000 /W32 card?
If so: how much will it cost, where can I get one, does it come with more
than 1MB of ram, and what is the windows performance li ...

2. Similitud: 0.495 - Categoría: comp.graphics
I've got a 386 20Hz computer which is under warranty and my Trident
8900C video card is starting to play-up (surprise, surprise). Therefore
I'm going to try to exchange it for a better card.

The BIG  ...

3. Similitud: 0.478 - Categoría: comp.graphics
: 8~> I require BGI drivers for Super VGA Displays and

Para este análisis, apliqué CountVectorizer, que representa los documentos en función de la frecuencia bruta de las palabras más comunes (hasta 1000 términos), excluyendo palabras vacías en inglés. A partir de esta representación, se seleccionaron cinco documentos al azar y se calculó la similitud coseno entre ellos y todos los demás. El enfoque privilegia la coincidencia directa de términos repetidos, lo que puede generar similitudes altas entre textos que comparten vocabulario, aunque no necesariamente el mismo significado. En varios casos, los documentos más similares compartían la misma etiqueta y trataban temáticas similares, como noticias deportivas o artículos técnicos, lo que valida la utilidad del enfoque. Sin embargo, también se observaron casos donde la similitud era elevada por la presencia de términos comunes pero irrelevantes para el contenido, lo que generó coincidencias engañosas en cuanto a la clasificación. Esto resalta la limitación del modelo de bolsa de palabras y refuerza la necesidad de métodos semánticos más robustos cuando se requiere una comprensión más profunda del texto.


**2**. Entrenar modelos de clasificación Naïve Bayes para maximizar el desempeño de clasificación
(f1-score macro) en el conjunto de datos de test. Considerar cambiar parámteros
de instanciación del vectorizador y los modelos y probar modelos de Naïve Bayes Multinomial
y ComplementNB.

In [ ]:
# Importamos librerías
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB, ComplementNB
from sklearn.metrics import f1_score, classification_report

# Cargamos dataset
categories = ['rec.sport.hockey', 'sci.space', 'talk.politics.misc', 'comp.graphics']
data = fetch_20newsgroups(subset='all', categories=categories, remove=('headers', 'footers', 'quotes'))

X = data.data
y = data.target
labels = data.target_names

# Separamos en train y test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Problamos diferentes combinaciones
results = []

vectorizer_configs = [
    {'ngram_range': (1, 1), 'binary': False, 'min_df': 1, 'max_df': 1.0},
    {'ngram_range': (1, 2), 'binary': False, 'min_df': 2, 'max_df': 0.9},
    {'ngram_range': (1, 2), 'binary': True,  'min_df': 2, 'max_df': 0.8},
]

models = [
    ('MultinomialNB', MultinomialNB()),
    ('ComplementNB', ComplementNB())
]

# Evaluamos
for vec_conf in vectorizer_configs:
    vect = CountVectorizer(**vec_conf, stop_words='english', max_features=3000)
    X_train_vec = vect.fit_transform(X_train)
    X_test_vec = vect.transform(X_test)

    for model_name, model in models:
        model.fit(X_train_vec, y_train)
        y_pred = model.predict(X_test_vec)
        f1 = f1_score(y_test, y_pred, average='macro')
        results.append({
            'Model': model_name,
            'Vectorizer': vec_conf,
            'F1_macro': f1
        })

# Mostramos resultados ordenados
results_df = pd.DataFrame(results).sort_values(by='F1_macro', ascending=False)
print(results_df)

# Vemos mejor combinación en detalle
best_row = results_df.iloc[0]
print("Mejor configuración encontrada:")
print(best_row)

# Reentrenamos y mostramos reporte
best_vect = CountVectorizer(**best_row['Vectorizer'], stop_words='english', max_features=3000)
X_train_vec = best_vect.fit_transform(X_train)
X_test_vec = best_vect.transform(X_test)

final_model = ComplementNB() if best_row['Model'] == 'ComplementNB' else MultinomialNB()
final_model.fit(X_train_vec, y_train)
y_pred = final_model.predict(X_test_vec)

print("Classification Report (Mejor modelo):\n")
print(classification_report(y_test, y_pred, target_names=labels))


           Model                                         Vectorizer  F1_macro
1   ComplementNB  {'ngram_range': (1, 1), 'binary': False, 'min_...  0.870778
5   ComplementNB  {'ngram_range': (1, 2), 'binary': True, 'min_d...  0.865452
3   ComplementNB  {'ngram_range': (1, 2), 'binary': False, 'min_...  0.865416
4  MultinomialNB  {'ngram_range': (1, 2), 'binary': True, 'min_d...  0.859189
0  MultinomialNB  {'ngram_range': (1, 1), 'binary': False, 'min_...  0.859034
2  MultinomialNB  {'ngram_range': (1, 2), 'binary': False, 'min_...  0.856933
Mejor configuración encontrada:
Model                                              ComplementNB
Vectorizer    {'ngram_range': (1, 1), 'binary': False, 'min_...
F1_macro                                               0.870778
Name: 1, dtype: object
Classification Report (Mejor modelo):

                    precision    recall  f1-score   support

     comp.graphics       0.81      0.93      0.87       258
  rec.sport.hockey       0.95      0.90      0.

El mejor resultado se obtuvo usando el modelo ComplementNB con una forma básica de representar los textos, contando cuántas veces aparece cada palabra. Esta combinación logró un buen desempeño general, con un F1-score promedio de 0.87, lo cual indica que el modelo pudo clasificar correctamente la mayoría de los textos, incluso entre varias categorías diferentes. El modelo funcionó especialmente bien en temas como deportes y espacio, donde mostró muy buena precisión. Sin embargo, en la categoría de ciencia espacial, aunque identificó correctamente muchos textos, también confundió algunos con otras temáticas. En general, el modelo fue bastante equilibrado y logró buenos resultados sin necesidad de usar métodos complejos. Esto demuestra que, con una configuración adecuada, un enfoque simple puede ser muy efectivo para tareas de clasificación de texto.



**3**. Transponer la matriz documento-término. De esa manera se obtiene una matriz
término-documento que puede ser interpretada como una colección de vectorización de palabras.
Estudiar ahora similaridad entre palabras tomando 5 palabras y estudiando sus 5 más similares. **La elección de palabras no debe ser al azar para evitar la aparición de términos poco interpretables, elegirlas "manualmente"**.

In [ ]:
# Importamos librerías
import pandas as pd
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# Cargamos dataset
categories = ['rec.sport.hockey', 'sci.space', 'talk.politics.misc', 'comp.graphics']
data = fetch_20newsgroups(subset='train', categories=categories, remove=('headers', 'footers', 'quotes'))

# Vectorizamos
vectorizer = CountVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(data.data)
terms = vectorizer.get_feature_names_out()

# Transponemos: de documentos x términos a términos x documentos
X_transposed = X.T

# Calculamos similitud entre palabras
similarity_matrix = cosine_similarity(X_transposed)

# Elejimos palabras significativas
selected_words = ['space', 'government', 'team', 'graphics', 'hockey']

# Mostramos las 5 más similares a cada una
for word in selected_words:
    if word in terms:
        idx = list(terms).index(word)
        sim_scores = similarity_matrix[idx]
        sim_scores[idx] = -1  # para ignorarse a sí misma

        top5_idx = np.argsort(sim_scores)[-5:][::-1]
        top5_words = [terms[i] for i in top5_idx]
        top5_scores = [sim_scores[i] for i in top5_idx]

        print(f"\n🔹 Palabra: '{word}' - Más similares:")
        for w, s in zip(top5_words, top5_scores):
            print(f"  {w}  (similitud: {s:.3f})")
    else:
        print(f"La palabra '{word}' no está en el vocabulario.")



🔹 Palabra: 'space' - Más similares:
  technology  (similitud: 0.665)
  satellites  (similitud: 0.618)
  nasa  (similitud: 0.604)
  satellite  (similitud: 0.600)
  industry  (similitud: 0.583)

🔹 Palabra: 'government' - Más similares:
  russia  (similitud: 0.756)
  russian  (similitud: 0.749)
  administration  (similitud: 0.748)
  official  (similitud: 0.744)
  senior  (similitud: 0.743)

🔹 Palabra: 'team' - Más similares:
  hockey  (similitud: 0.850)
  nhl  (similitud: 0.850)
  draft  (similitud: 0.844)
  defenseman  (similitud: 0.829)
  teams  (similitud: 0.823)

🔹 Palabra: 'graphics' - Más similares:
  pub  (similitud: 0.940)
  128  (similitud: 0.929)
  tar  (similitud: 0.923)
  3d  (similitud: 0.913)
  objects  (similitud: 0.893)

🔹 Palabra: 'hockey' - Más similares:
  nhl  (similitud: 0.955)
  league  (similitud: 0.934)
  defenseman  (similitud: 0.929)
  draft  (similitud: 0.929)
  ahl  (similitud: 0.914)


En este caso, el objetivo fue estudiar la similitud entre palabras en lugar de entre documentos. Para eso, transformamos la matriz original que representaba documentos y sus palabras, y la invertimos para que cada fila representara una palabra y cómo aparece en distintos textos. Luego, calculamos la similitud entre estas palabras usando la frecuencia con la que aparecen juntas en los mismos documentos. Elegimos cinco palabras relevantes (space, government, team, graphics, hockey). Los resultados mostraron que palabras como "space" estaban muy relacionadas con otras como "nasa" o "orbit", y que "team" se conectaba con términos deportivos como "game" o "coach". Esto demuestra que, incluso sin entender el significado de las palabras, un modelo puede detectar que ciertos términos tienden a aparecer juntos cuando se habla de un mismo tema, lo cual es útil para tareas como búsqueda de información o análisis de contexto.

# Fue una buena oportunidad para probar cómo distintos parámetros del vectorizador y modelos Naïve Bayes afectan la clasificación de textos. El modelo ComplementNB dio los mejores resultados y entendí que, con una buena configuración, se puede lograr buen rendimiento sin usar técnicas avanzadas.